# 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# 데이터 불러오기

In [9]:
data_path: str = "~/house/data"
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [10]:
# 금리, 지하철, 학교, 공원 정보 불러오기
interest_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
subway_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))
school_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
park_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))

In [12]:
train_data.head()

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0


In [13]:
test_data.head()

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age
0,0,84.961,202404,12,1,14,2016,36.965423,127.048779,8
1,1,59.900,202404,13,0,4,1997,36.963105,127.040678,27
2,2,39.270,202404,29,0,5,1990,36.957089,127.047449,34
3,3,39.270,202405,3,0,1,1990,36.957089,127.047449,34
4,4,46.980,202406,2,0,4,1990,36.957089,127.047449,34


In [14]:
interest_data

,year_month,interest_rate
0,202405,3.56
1,202404,3.54
2,202403,3.59
3,202402,3.62
4,202401,3.66
...,...,...
61,201904,1.85
62,201903,1.94
63,201902,1.92
64,201901,1.99


In [22]:
merged_train = pd.merge(train_data, interest_data, left_on='contract_year_month', right_on='year_month', how='left')
merged_train = merged_train.drop(columns=['year_month'])
print(merged_train.head())

   index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0  84.9981               201906            25              2      9   
1      1  84.9981               202003            26              2     20   
2      2  84.9981               202003            28              2      8   
3      3  59.3400               201907            15              2      1   
4      4  59.8100               201904            12              2      6   

   built_year   latitude   longitude  age  deposit  interest_rate  
0        2019  37.054314  127.045216    0  17000.0           1.78  
1        2019  37.054314  127.045216    1  23000.0           1.26  
2        2019  37.054314  127.045216    1  23000.0           1.26  
3        1986  36.964647  127.055847   33   5000.0           1.68  
4        1995  36.972390  127.084514   24   1800.0           1.85  


In [24]:
merged_test = pd.merge(test_data, interest_data, left_on='contract_year_month', right_on='year_month', how='left')
merged_test = merged_test.drop(columns=['year_month'])
print(merged_test.head())

   index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0   84.961               202404            12              1     14   
1      1   59.900               202404            13              0      4   
2      2   39.270               202404            29              0      5   
3      3   39.270               202405             3              0      1   
4      4   46.980               202406             2              0      4   

   built_year   latitude   longitude  age  interest_rate  
0        2016  36.965423  127.048779    8           3.54  
1        1997  36.963105  127.040678   27           3.54  
2        1990  36.957089  127.047449   34           3.54  
3        1990  36.957089  127.047449   34           3.56  
4        1990  36.957089  127.047449   34            NaN  


In [21]:
merged_train.to_csv('train_data_merged_interest.csv', index=False, encoding='utf-8-sig')

In [ ]:
merged_test.to_csv('test_data_merged_interest.csv', index=False, encoding='utf-8-sig')